In [1]:
import os
from pymystem3.mystem import Mystem
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from string import punctuation, digits
punctuation = set(punctuation + '«»—–…“”\n\t' + digits)

In [2]:
### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов

main_dir = '/media/zu_ann/OS/Users/zu_ann/Yandex.Disk/HSE/IR/Friends'
files_list = []

for root, dirs, files in os.walk(main_dir):
    for dr in dirs:
        files_list += [main_dir + '/' + dr + '/' + file for file in os.listdir(main_dir + '/' + dr)]

In [3]:
### _check : в коллекции должно быть 165 файлов

len(files_list)

165

In [4]:
def preprocess_words(text):
    m = Mystem()
    table = str.maketrans({ch: ' ' for ch in punctuation})
    
    tokenized = word_tokenize(text.replace('\ufeff', '').lower().translate(table))
    
    return [m.lemmatize(word)[0] for word in tokenized], len(tokenized)

In [5]:
def preprocess_files(file):
    with open(file, 'r', encoding='utf-8') as f:
        words_list, text_length = preprocess_words(f.read())
        document_length[files_list.index(file)] = text_length
        
        return words_list

In [6]:
def get_inverted_index(files_list) -> dict:
    """
    Create inverted index by input doc collection and count the length of each document 
    :return: inverted index
    """
    inverted_index = defaultdict(list)
    global document_length
    document_length = [None] * len(files_list)

    for file in files_list:
        for word in preprocess_files(file):
            inverted_index[word].append(files_list.index(file))
    
    return inverted_index

In [7]:
inverted_index = get_inverted_index(files_list)

In [8]:
def preprocess_query(input_text, operations):
    stack = []
    output = []
    table = str.maketrans({'(': '( ', ')': ' )'})

    query = input_text.lower().translate(table).split()

    for elem in query:
        
        if elem == '(':
            stack.append(elem)
        
        elif elem in operations:
            if len(stack) > 0 and stack[-1] != '(':
                output.append(stack[-1])
                stack[-1] = elem
            else:
                stack.append(elem)
        
        elif elem == ')':
            if stack[-1] == '(':
                continue
            else:
                k = stack[-1]
                while k != '(':
                    output.append(stack.pop(-1))
                    k = stack[-1]
                stack.pop(-1)
    
        else:
            output.append(elem)

    if len(stack) == 1:
        output = output + stack
    elif len(stack) > 1:
        output = output + stack[::-1]

    return output

In [9]:
### напишите функцию булева поиска по построенной матрице

def boolean_search(input_text, inverted_index, collection_size):
    """
    Produces a Boolean search according with the inverted index
    :return: list of first 5 relevant documents
    """
    relevant_documents = []
    operations = ['&', 'или', 'не']
    query = preprocess_query(input_text, operations)
    
    for i, elem in enumerate(query):
        if elem not in operations:
            relevant_documents.append(elem)
    
        elif elem == 'не':
            a = relevant_documents.pop()
            if type(a) != set:
                a = set(inverted_index[a])
            relevant_documents.append(set(range(collection_size)) - a)
    
        else:
            a = relevant_documents.pop()
            if type(a) != set:
                a = set(inverted_index[a])
        
            b = relevant_documents.pop()
            if type(b) != set:
                b = set(inverted_index[b])

        if elem == '&':
                relevant_documents.append(a & b)
        elif elem == 'или':
            relevant_documents.append(a | b)
    
    return relevant_documents[:5]

In [10]:
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джоуи & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джоуи) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джоуи & (НЕ Росс)'
]

In [11]:
print(boolean_search(input_text[0], inverted_index, len(files_list)))

[{6, 136, 137, 10, 138, 13, 14, 142, 144, 145, 146, 147, 148, 149, 154, 27, 156, 160, 162, 163, 164, 51, 68, 113, 117, 119, 122, 123, 125}]


In [12]:
print(boolean_search(input_text[1], inverted_index, len(files_list)))

[{0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 142, 144, 145, 146, 147, 148, 21, 22, 149, 150, 154, 27, 155, 29, 156, 160, 162, 134, 163, 164, 39, 136, 137, 51, 54, 138, 68, 70, 112, 113, 114, 115, 117, 119, 120, 122, 123, 125}]


In [13]:
print(boolean_search(input_text[2], inverted_index, len(files_list)))

[set()]


С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?

In [14]:
for word in sorted(inverted_index, key=lambda x: len(inverted_index[x]), reverse=True)[:1]:
    print(word, len(inverted_index[word]))

я 18530


- какое самым редким?

In [15]:
for word in sorted(inverted_index, key=lambda x: len(inverted_index[x]), reverse=False)[:1]:
    print(word, len(inverted_index[word]))

реалистичный 1


- какой набор слов есть во всех документах коллекции?

In [16]:
for word in inverted_index:
    if set(range(165)) == set(inverted_index[word]):
        print(word)

если
не
что
мы
быть
он
а
то
и
так
на
как
в
но
у
ты
этот
такой
просто
да
ну
сказать
еще
о
знать
это
она
думать
они
нет
все
хорошо
хотеть
мой
я
с


2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   

In [17]:
def count_popular_season(inverted_index, name):
    seasons = [[i for i in range(20)] + [0], 
               [i for i in range(20, 45)] + [0], 
               [i for i in range(45, 66)] + [0], 
               [i for i in range(66, 92)] + [0],
               [i for i in range(92, 116)] + [0],
               [i for i in range(116, 141)] + [0],
               [i for i in range(141, 165)] + [0]]
    
    for ind in inverted_index[name]:
        for i, season in enumerate(seasons):
            if ind in season:
                seasons[i][-1] += 1
    
    max_times = [0, 0]
    for i, season in enumerate(seasons):
        if season[-1] > max_times[1]:
            max_times[0] = i + 1
            max_times[1] = season[-1]
    
    return max_times[0]

In [18]:
count_popular_season(inverted_index, 'чендлер')

6

In [19]:
count_popular_season(inverted_index, 'моника')

7

- кто из главных героев статистически самый популярный? 

In [20]:
persons = ['чендлер', 'джоуи', 'моника', 'фиби', 'росс', 'рэйчел']
for person in persons:
    print(person, len(inverted_index[person]))

чендлер 724
джоуи 682
моника 684
фиби 575
росс 1016
рэйчел 237


Самый популярный: Росс.

In [21]:
### реализуйте эту функцию ранжирования 

from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    score = log((N - n + 0.5) / (n + 0.5)) * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))
    return score

In [22]:
def compute_sim(lemma, inverted_index, document_length):
    """
    Compute similarity score between word in search query and all document from collection
    :return: score
    """
    doc_list = inverted_index[lemma]
    relevance_score = {}
    avgdl = sum(document_length) / len(document_length)
    N = len(document_length)
    
    for doc in range(N):    
        qf = Counter(inverted_index[lemma])[doc]
        relevance_score[doc] = score_BM25(qf, document_length[doc], avgdl,
                                          2.0, 0.75, N, len(set(inverted_index[lemma])))
    return relevance_score

In [23]:
def get_search_result(query, inverted_index, document_length, num_res):
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    relevance_dict = defaultdict(float)
    lemmas, _ = preprocess_words(query)
    
    for lemma in lemmas:
        score = compute_sim(lemma, inverted_index, document_length)
        for elem in score:
            relevance_dict[elem] += score[elem]
            
    result = sorted(relevance_dict, key=relevance_dict.get, reverse=True)[:num_res]
    
    return [(files_list[ind].split('/Friends/Friends - ')[1], relevance_dict[ind]) for ind in result]

In [24]:
res = get_search_result('рождественские каникулы', inverted_index, document_length, 10)
for elem in res:
    print('{}: {}'.format(elem[0], elem[1]))

season 7/Friends - 7x10 - The One With The Holiday Armadillo.ru.txt: 9.976228146378661
season 6/Friends - 6x19 - The One With Joey's Fridge.ru.txt: 7.917781782429753
season 3/Friends - 3x10 - The One Where Rachel Quits.ru.txt: 5.712885994666163
season 2/Friends - 2x09 - The One With Phoebe's Dad.ru.txt: 4.849771642565855
season 1/Friends - 1x17 - The One With Two Parts (2).ru.txt: 4.107940734356043
season 4/Friends - 4x03 - The One With The 'Cuffs.ru.txt: 4.07543546317729
season 1/Friends - 1x16 - The One With Two Parts (1).ru.txt: 4.051197790928666
season 4/Friends - 4x10 - The One With The Girl From Poughkeepsie.ru.txt: 3.974191647399251
season 6/Friends - 6x12 - The One With The Joke.ru.txt: 3.442200369543739
season 6/Friends - 6x09 - The One Where Ross Got High.ru.txt: 3.380802007282508
